In [1]:
import yaml
import os
import pandas as pd
totalfiles = './newrelic-quickstarts/quickstarts/'

In [2]:
with open('./newrelic-quickstarts/_template/alerts/static-alert.yml','r') as f:
    data = yaml.load(f)
data

/home/weijunfei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


{'name': 'Name of this alert condition',
 'details': 'Describe the purpose of this alert condition.\nYou can use multiple lines.\nTry to be as descriptive as possible and provide "The Why".\n\n',
 'type': 'STATIC',
 'nrql': {'query': 'FROM SystemSample SELECT average(cpuPercent) FACET entityName'},
 'valueFunction': 'SINGLE_VALUE | SUM',
 'terms': [{'priority': 'CRITICAL',
   'operator': 'ABOVE | BELOW | EQUALS',
   'threshold': 90,
   'thresholdDuration': 600,
   'thresholdOccurrences': 'ALL | AT_LEAST_ONCE'},
  {'priority': 'WARNING',
   'threshold': 80,
   'operator': 'ABOVE | BELOW | EQUALS',
   'thresholdDuration': 300,
   'thresholdOccurrences': 'ALL | AT_LEAST_ONCE'}],
 'expiration': {'closeViolationsOnExpiration': 'true | false',
  'openViolationOnExpiration': 'true | false',
  'expirationDuration': None},
 'signal': {'aggregationWindow': 60,
  'evaluationOffset': 3,
  'fillOption': 'LAST_VALUE | NONE | STATIC',
  'fillValue': None},
 'runbookUrl': None,
 'description': None,
 

In [3]:
with open('./newrelic-quickstarts/_template/alerts/baseline-alert.yml','r') as f:
    data = yaml.load(f)
data

/home/weijunfei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


{'name': 'Name of this alert condition',
 'details': 'Describe the purpose of this alert condition.\nYou can use multiple lines.\nTry to be as descriptive as possible and provide "The Why".\n\n',
 'type': 'BASELINE',
 'nrql': {'query': 'FROM Metric SELECT average(metric.name) FACET entity.name'},
 'baselineDirection': 'LOWER_ONLY | UPPER_AND_LOWER | UPPER_ONLY',
 'terms': [{'priority': 'CRITICAL',
   'threshold': 3,
   'thresholdDuration': 60,
   'thresholdOccurrences': 'ALL | AT_LEAST_ONCE'},
  {'priority': 'WARNING',
   'threshold': 1,
   'thresholdDuration': 300,
   'thresholdOccurrences': 'ALL | AT_LEAST_ONCE'}],
 'expiration': {'closeViolationsOnExpiration': 'true | false',
  'openViolationOnExpiration': 'true | false',
  'expirationDuration': None},
 'signal': {'evaluationOffset': 3,
  'fillOption': 'LAST_VALUE | NONE | STATIC',
  'fillValue': None},
 'runbookUrl': None,
 'description': None,
 'violationTimeLimitSeconds': 86400}

## 解析文件夹
选取含有alert文件的文件名

In [4]:
def getpath(pathname, filename):
    return os.path.join(pathname, filename)

如果 'alert' 在 文件中，那么就返回含'alert'的文件名；\
如果 'alert' 不在 文件中，同时已经到最后一层，返回None;\
除此之外（即还未到最后一层），那就到下一层继续上述判断，通过递归的方式

In [5]:
def getalert(pathname, filename):
    num = []
    config = ['config.yml', 'config.yaml']
    condition1 = [i for i in config if i in os.listdir(getpath(pathname, filename))]
    if len(condition1) != 0  and 'alerts' not in os.listdir(getpath(pathname, filename)) :
        return None
    elif len(condition1) != 0 and 'alerts' in os.listdir(getpath(pathname, filename)) :
        return getpath(getpath(pathname, filename), 'alerts')
    else:
        for i in os.listdir(getpath(pathname, filename)):
            num.append(getalert(getpath(pathname, filename), i))
        return num
        
# print(getalert(totalfiles, 'android')) # 一级，不含有alerts
# print(getalert(totalfiles, 'fastly')) # 一级，含有alerts
# print(getalert(totalfiles, 'golang')) # 多级，含有alerts
# print(getalert(totalfiles, 'ads')) # 多级，不含有alerts

In [6]:
# 将所有含alerts的文件路径汇总，但是里面存在None值
filenames = os.listdir(totalfiles)
hasalerts = []
for i in filenames:
    hasalerts.append(getalert(totalfiles, i))

去除None值

In [7]:
nums = []
for i in hasalerts:
    if isinstance(i,list):
        if any(i):
            nums.append([j for j in i if j != None])
       
    else:
        if not pd.isnull(i):
            nums.append([i])       

In [8]:
nums = [j for i in nums for j in i]
finalalerts = []
for i in nums:
    if isinstance(i, list):
        if all(i):
            finalalerts.append([j for j in i])
    else:
        finalalerts.append([i])

In [9]:
finalalertsfilepath = [j for i in finalalerts for j in i]

In [10]:
len(finalalertsfilepath)

184

## 读取yaml文件

In [11]:
import glob
files = glob.glob(finalalertsfilepath[0]+'/*.yml')

In [12]:
files

['./newrelic-quickstarts/quickstarts/c/alerts/CPUUtilization.yml',
 './newrelic-quickstarts/quickstarts/c/alerts/HighMemoryUsage.yml',
 './newrelic-quickstarts/quickstarts/c/alerts/LowApdexScore.yml',
 './newrelic-quickstarts/quickstarts/c/alerts/TransactionError.yml']

In [13]:
# 单个文件
f = open(files[0], 'r')
data = yaml.load_all(f, Loader=yaml.FullLoader)
resList = []
for doc in data:
    resList.append(doc)
[i for i in resList if not i == None]

[{'name': 'High CPU Utilization',
  'details': 'This alert is triggered when the CPU Utilization is above 90%.\n',
  'type': 'STATIC',
  'nrql': {'query': "SELECT rate(sum(apm.service.cpu.usertime.utilization), 1 second) * 100 as cpuUsage FROM Metric WHERE appName like '%'"},
  'valueFunction': 'SINGLE_VALUE',
  'terms': [{'priority': 'CRITICAL',
    'operator': 'ABOVE',
    'threshold': 90,
    'thresholdDuration': 300,
    'thresholdOccurrences': 'ALL'}],
  'violationTimeLimitSeconds': 86400}]

### 读取数据

In [14]:
def readyaml(file):
    f = open(file, 'r')
    datas = yaml.load_all(f, Loader = yaml.FullLoader)
    return [i for i in [doc for doc in datas] if not i == None]
[j for i in [readyaml(file) for file in files] for j in i]

[{'name': 'High CPU Utilization',
  'details': 'This alert is triggered when the CPU Utilization is above 90%.\n',
  'type': 'STATIC',
  'nrql': {'query': "SELECT rate(sum(apm.service.cpu.usertime.utilization), 1 second) * 100 as cpuUsage FROM Metric WHERE appName like '%'"},
  'valueFunction': 'SINGLE_VALUE',
  'terms': [{'priority': 'CRITICAL',
    'operator': 'ABOVE',
    'threshold': 90,
    'thresholdDuration': 300,
    'thresholdOccurrences': 'ALL'}],
  'violationTimeLimitSeconds': 86400},
 {'name': 'Memory Usage',
  'details': 'This alert is triggered when Memory usage is above 90%\n',
  'type': 'STATIC',
  'nrql': {'query': 'SELECT average(memoryUsedBytes/memoryTotalBytes) * 100 FROM SystemSample'},
  'valueFunction': 'SINGLE_VALUE',
  'terms': [{'priority': 'CRITICAL',
    'operator': 'ABOVE',
    'threshold': 90,
    'thresholdDuration': 300,
    'thresholdOccurrences': 'ALL'}],
  'violationTimeLimitSeconds': 86400},
 {'name': 'Apdex Score',
  'details': 'This alert is trigge

### 处理数据

In [15]:
def handledata(results):
    results1 = {}
    for keys,values in results.items():
        if isinstance(values, dict):
            for key,value in values.items():
                results1['{}_{}'.format(keys, key)] = value
        elif isinstance(values, list):
            for value in values:
                if isinstance(value, dict):
                    if 'priority' in value.keys():
                        if value['priority'] == 'CRITICAL':
                            for subkey,subvalue in value.items():
                                results1['{}_{}_{}'.format(keys, 'critical', subkey)] = subvalue
                        elif value['priority'] == 'WARNING':
                            for subkey,subvalue in value.items():
                                results1['{}_{}_{}'.format(keys, 'WARNING', subkey)] = subvalue            
                else:
                    print(value)
        else:
            results1[keys] = values
    return results1
results = readyaml(files[0])[0]
handledata(results)

{'name': 'High CPU Utilization',
 'details': 'This alert is triggered when the CPU Utilization is above 90%.\n',
 'type': 'STATIC',
 'nrql_query': "SELECT rate(sum(apm.service.cpu.usertime.utilization), 1 second) * 100 as cpuUsage FROM Metric WHERE appName like '%'",
 'valueFunction': 'SINGLE_VALUE',
 'terms_critical_priority': 'CRITICAL',
 'terms_critical_operator': 'ABOVE',
 'terms_critical_threshold': 90,
 'terms_critical_thresholdDuration': 300,
 'terms_critical_thresholdOccurrences': 'ALL',
 'violationTimeLimitSeconds': 86400}

### 汇总数据

In [16]:
results = []
for result in [j for i in [readyaml(file) for file in files] for j in i]:
    results.append(handledata(result))

In [36]:
def getdata(final):
    files = glob.glob(final+'/*.yml')
    addname = []
    for file in files:
        origin = readyaml(file)
        for i in origin:
            i['filename'] = file.split('/',3)[-1]
        addname.append(origin)
        
    results = []
    for result in [j for i in addname for j in i]:
        results.append(handledata(result))
    return results

In [37]:
finals = [getdata(final) for final in finalalertsfilepath]
len([j for i in finals for j in i])

561

In [38]:
data = pd.DataFrame([j for i in finals for j in i])

In [39]:
data.to_excel('./new relic.xlsx', index = False)

In [40]:
data.columns

Index(['name', 'details', 'type', 'nrql_query', 'valueFunction',
       'terms_critical_priority', 'terms_critical_operator',
       'terms_critical_threshold', 'terms_critical_thresholdDuration',
       'terms_critical_thresholdOccurrences', 'violationTimeLimitSeconds',
       'filename', 'terms_WARNING_priority', 'terms_WARNING_operator',
       'terms_WARNING_threshold', 'terms_WARNING_thresholdDuration',
       'terms_WARNING_thresholdOccurrences', 'signal_aggregationWindow',
       'signal_evaluationOffset', 'signal_fillOption', 'signal_fillValue',
       'baselineDirection', 'expiration_closeViolationsOnExpiration',
       'expiration_openViolationOnExpiration', 'expiration_expirationDuration',
       'runbookUrl', 'description', 'expectedGroups'],
      dtype='object')